In [ ]:
from pyspark import SparkContext, SparkConf 

In [56]:
# базовую конфигурация, которая осуществляется через специальный класс SparkConf
# чтобы воспользоваться объектом SparkConf, вызываем методами setAppName() и setMaster()
conf = SparkConf().setAppName("lab1").setMaster('yarn')

# передаем объект в конструктор объекта SparkContext
sc = SparkContext(conf=conf)

In [57]:
!hadoop fs -put * . # перемещаем файлы

put: `lab1.ipynb': File exists
put: `station.csv': File exists
put: `trip.csv': File exists
put: `warandsociety.txt': File exists


In [58]:
!hadoop fs -ls # провереряем перенеслись ли файлы

Found 5 items
drwxr-xr-x   - kasparov kasparov          1 2023-12-21 20:41 .sparkStaging
-rwxr-xr-x   3 kasparov kasparov      16216 2023-12-21 01:02 lab1.ipynb
-rwxr-xr-x   3 kasparov kasparov       5647 2023-12-21 01:02 station.csv
-rwxr-xr-x   3 kasparov kasparov   80208848 2023-12-21 01:02 trip.csv
-rwxr-xr-x   3 kasparov kasparov    5315699 2023-12-21 01:02 warandsociety.txt


In [61]:
# считываем текстовый файл из HDFS, и возвращаем его как RDD строк.
warandpeace = sc.textFile("warandsociety.txt")

# выводим количество строк файла
print(warandpeace.count()) 

12851


In [62]:
# Считываем первые 10 строк файла warandsociety.txt.
warandpeace.take(10)

['Лев Николаевич Толстой',
 'Война и мир. Книга 1',
 '',
 'Война и мир – 1',
 '',
 ' ',
 ' http://www.lib.ru',
 '',
 'Аннотация ',
 '']

In [66]:
# Ищем строки содержащие слово "война" и выводим первую из них
linesWithWar = warandpeace.filter(lambda x: "война" in x)
linesWithWar.first()

"– Еh bien, mon prince. Genes et Lucques ne sont plus que des apanages, des поместья, de la famille Buonaparte. Non, je vous previens, que si vous ne me dites pas, que nous avons la guerre, si vous vous permettez encore de pallier toutes les infamies, toutes les atrocites de cet Antichrist (ma parole, j'y crois) – je ne vous connais plus, vous n'etes plus mon ami, vous n'etes plus мой верный раб, comme vous dites. [Ну, что, князь, Генуа и Лукка стали не больше, как поместьями фамилии Бонапарте. Нет, я вас предупреждаю, если вы мне не скажете, что у нас война, если вы еще позволите себе защищать все гадости, все ужасы этого Антихриста (право, я верю, что он Антихрист) – я вас больше не знаю, вы уж не друг мой, вы уж не мой верный раб, как вы говорите.] Ну, здравствуйте, здравствуйте. Je vois que je vous fais peur, [Я вижу, что я вас пугаю,] садитесь и рассказывайте."

In [68]:
# Замер времени выполнения команды
def time(f):    
    import time
    t = time.process_time()
    f()
    print(f"Elapsed time: {int((time.process_time() - t)*1e9)} ns")


In [70]:
linesWithWar.cache()
time(lambda: linesWithWar.count())
time(lambda: linesWithWar.count())

Elapsed time: 8222722 ns
Elapsed time: 15242321 ns


In [74]:
# Поиск гистограммы слов
wordCounts = linesWithWar.flatMap(lambda line: line.split(" ")).map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)

In [76]:
# Сохранение гистограммы в txt файл
wordCounts.saveAsTextFile("warandpeace_histogram_2.txt")

In [78]:
temp = sc.textFile("warandpeace_histogram_2.txt")
temp.take(10)

["('–', 92)",
 "('bien,', 1)",
 "('Genes', 1)",
 "('et', 4)",
 "('Lucques', 1)",
 "('ne', 4)",
 "('plus', 3)",
 "('apanages,', 1)",
 "('de', 13)",
 "('la', 6)"]

In [81]:
# Операции с множествами
# Инициализация двух множеств
a = sc.parallelize([1,2,3,4])
b = sc.parallelize([3,4,6,7])



In [82]:
a.union(b).distinct().collect() # объединение a и b, удаление дубликатов

[4, 1, 2, 6, 3, 7]

In [85]:
a.intersection(b).collect() # пересечение

[4, 3]

In [84]:
a.subtract(b).collect() # разность

[1, 2]

In [86]:
sc.stop()